In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from PY_PAD_on_sphere_library import calculate_attributions_from_xarrays
from postprocess import     aggregate_transportplan_at_gridpoints, postprocess_residue_df, get_latlon_df

## Read sample data

Both sample fields are on a 0.25 deg grid

In [2]:
fcst = xr.open_dataarray("PY_PAD_on_sphere_example_field_A.nc")
obs = xr.open_dataarray("PY_PAD_on_sphere_example_field_B.nc")

### Reshape the data to have a single "gridpoint" dimension instead of "lat" and "lon"

In [4]:
fcst = fcst.stack(stack=("lat", "lon"))
obs = obs.stack(stack=("lat", "lon"))
fcst = fcst.assign_coords(gridpoint=('stack', range(fcst.sizes['stack']))).swap_dims({"stack": "gridpoint"}).drop_vars("stack")
obs = obs.assign_coords(gridpoint=('stack', range(obs.sizes['stack']))).swap_dims({"stack": "gridpoint"}).drop_vars("stack")

In [5]:
fcst

<xarray.DataArray 'precipitation' (gridpoint: 1038240)> Size: 4MB
array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)
Coordinates:
    lat        (gridpoint) float32 4MB -90.0 -90.0 -90.0 ... 90.0 90.0 90.0
    lon        (gridpoint) float32 4MB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * gridpoint  (gridpoint) int64 8MB 0 1 2 3 ... 1038236 1038237 1038238 1038239

### Compute grid-area (used internally to convert precipitation from heigh in mm to volume in m^3)

In [6]:
dlat = 0.25 # in deg
dlon = 0.25 # in deg
Earth_radius = 6371 # in km
area = (
    np.deg2rad(dlat) * Earth_radius
    * np.deg2rad(dlon) * Earth_radius * np.cos(np.deg2rad(fcst.lat))
) # in km^2

## Compute PAD-on-sphere attributions

In [12]:
attr_df, nonattr_ds = calculate_attributions_from_xarrays(fcst, obs, area)

----- preprocessing: 0.0117621 s
----- kdtree construction: 0.385985 s
----- attribution: 7.41096 s


In [13]:
attr_df

,distance_m,volume_m3,gridpoint_fcst,gridpoint_obs
0,0,2.572534e+01,1913,1913
1,0,2.572534e+01,1914,1914
2,0,2.572534e+01,1915,1915
3,0,2.572534e+01,1916,1916
4,0,2.572534e+01,1917,1917
...,...,...,...,...
1331106,3013998,4.724616e+06,713561,618617
1331107,3017783,4.697494e+05,301571,210722
1331108,3018263,3.947679e+05,301571,174951
1331109,3023937,4.545606e+04,303015,313231


In [14]:
nonattr_ds

<xarray.Dataset> Size: 33MB
Dimensions:    (gridpoint: 1038240)
Coordinates:
    lat        (gridpoint) float64 8MB -90.0 -90.0 -90.0 ... 90.0 90.0 90.0
    lon        (gridpoint) float64 8MB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * gridpoint  (gridpoint) int64 8MB 0 1 2 3 ... 1038236 1038237 1038238 1038239
Data variables:
    error      (gridpoint) float64 8MB -0.0 -0.0 -0.0 -0.0 ... -0.0 -0.0 -0.0

In [10]:
latlon_df = get_latlon_df(obs)

In [15]:
distance_ds = aggregate_transportplan_at_gridpoints(attr_df, latlon_df).to_xarray()